In [1]:
#import the regnskaber here

from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.types import StructType,StructField,StringType,MapType,IntegerType,ArrayType,DateType,DoubleType
import numpy as np
import re
import ast
import sys
import operator
import matplotlib.pyplot as plt
from datetime import datetime

SQLContext.newSession(sqlContext)
cleanCsvsPath = "/home/svanhmic/workspace/Python/Erhvervs/data/regnskabsdata/sparkdata/csv"
cleanParquePath = "/home/svanhmic/workspace/Python/Erhvervs/data/regnskabsdata/sparkdata/parquet"

In [ ]:
df1 = sqlContext.read.parquet(cleanParquePath+"/regnskaber1.parquet")

In [4]:
df1.show(10)

+--------------------+----+----+----+-------+----------+----------------+----------+-----------+--------------------+----------+--------------+
|                Name| Dec|Prec|Lang|unitRef|contextRef|EntityIdentifier|     Start|End_Instant|               Value|Dimensions|originalLength|
+--------------------+----+----+----+-------+----------+----------------+----------+-----------+--------------------+----------+--------------+
|gsd:ReportingPeri...|null|null|null|   null|  16993085|        16993085|2013-10-01| 2014-09-30|          2013-10-01|        []|            10|
|gsd:ReportingPeri...|null|null|null|   null|  16993085|        16993085|2013-10-01| 2014-09-30|          2014-09-30|        []|            10|
|mrv:DescriptionOf...|null|null|  da|   null|  16993085|        16993085|2013-10-01| 2014-09-30|"Siemens er en in...|        []|         27116|
|sob:StatementByEx...|null|null|  da|   null|  16993085|        16993085|2013-10-01| 2014-09-30|"B.1 Ledelsens re...|        []|        

In [ ]:
df2 = sqlContext.read.parquet(cleanParquePath+"/regnskaber2.parquet")

In [5]:
df2.show(10)

+--------------------+----+----+----+-------+----------+----------------+----------+-----------+--------------------+----------+--------------+
|                Name| Dec|Prec|Lang|unitRef|contextRef|EntityIdentifier|     Start|End_Instant|               Value|Dimensions|originalLength|
+--------------------+----+----+----+-------+----------+----------------+----------+-----------+--------------------+----------+--------------+
|gsd:InformationOn...|null|null|  da|   null|  36453443|        36453443|2014-12-02| 2015-12-31|     &#197;rsrapport|        []|            15|
|gsd:DateOfGeneral...|null|null|null|   null|  36453443|        36453443|2014-12-02| 2015-12-31|          2016-06-07|        []|            10|
|gsd:NameAndSurnam...|null|null|  da|   null|  36453443|        36453443|2014-12-02| 2015-12-31|        Michael Bech|        []|            12|
|gsd:NameOfReporti...|null|null|  da|   null|  36453443|        36453443|2014-12-02| 2015-12-31|M.S. EJENDOMME 20...|        []|        

In [6]:
df3 = sqlContext.read.parquet(cleanParquePath+"/regnskaber3.parquet")
df3.show(10)

+--------------------+----+----+----+-------+----------+----------------+----------+-----------+--------------------+----------+--------------+
|                Name| Dec|Prec|Lang|unitRef|contextRef|EntityIdentifier|     Start|End_Instant|               Value|Dimensions|originalLength|
+--------------------+----+----+----+-------+----------+----------------+----------+-----------+--------------------+----------+--------------+
|fsa:ClassOfReport...|null|null|null|   null|  20438401|        20438401|2013-01-01| 2013-12-31|Regnskabsklasse C...|        []|            40|
|gsd:ReportingPeri...|null|null|null|   null|  20438401|        20438401|2013-01-01| 2013-12-31|          2013-01-01|        []|            10|
|gsd:ReportingPeri...|null|null|null|   null|  20438401|        20438401|2013-01-01| 2013-12-31|          2013-12-31|        []|            10|
|         fsa:Revenue|  -3|null|null|    DKK|  20438401|        20438401|2013-01-01| 2013-12-31|           7,500,000|        []|        

In [7]:
dfTotal = df1.unionAll(df2)
dfTotal.count()

64350877

In [8]:
allThreeDf = dfTotal.unionAll(df3)
allThreeDf.count()

allThreeDf.write.parquet(cleanParquePath+"/alleregnskaber.parquet","overwrite")

In [9]:
#extract all rows with debt in them
debtCols = allThreeDf.columns+["DebtNames"]

debtDf = (allThreeDf
 .select("*",F.regexp_extract("Name",".*debt|Debt.*",0).alias("DebtNames"))
 .filter((F.col("DebtNames") != ""))
)

In [10]:
debtDf.write.parquet(cleanParquePath+"/regnskaberDebt.parquet","overwrite")

In [11]:
groupedNamesDf = allThreeDf.groupBy("Name").count()

In [12]:
groupedNamesDf.orderBy(F.col("count").desc()).show(50,truncate=False)

+------------------------------------------------+-------+
|Name                                            |count  |
+------------------------------------------------+-------+
|fsa:ProfitLoss                                  |2483478|
|e:Equity                                        |1887451|
|e:ProfitLoss                                    |1435357|
|fsa:Equity                                      |1199071|
|fsa:Assets                                      |751870 |
|fsa:LiabilitiesAndEquity                        |716450 |
|fsa:ContributedCapital                          |688128 |
|fsa:RetainedEarnings                            |679204 |
|fsa:CurrentAssets                               |670844 |
|fsa:LiabilitiesOtherThanProvisions              |669518 |
|fsa:ShorttermLiabilitiesOtherThanProvisions     |651598 |
|fsa:CashAndCashEquivalents                      |610169 |
|fsa:ProfitLossFromOrdinaryOperatingActivities   |605220 |
|fsa:OtherShorttermPayables                      |594051